# scratch

In [1]:
import zarr
import numpy as np
from cloudvolume import CloudVolume, Skeleton

In [4]:
z = zarr.open('/Users/thomasathey/Documents/jovolab/neurodata_infrastructure_tips/neuroglancer-zarr/test.zarr', mode='w', shape=(100, 100,100),chunks=(50, 50, 100), dtype='int')
z[:,:,:] = 0
z[45:55,45:55,45:55] = 1

python cors_webserver.py -d "/Users/thomasathey/Documents/jovolab/neurodata_infrastructure_tips/neuroglancer-zarr/" -p 9010

In [35]:

ng_path = "precomputed://file:///Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/sample/ng/frags"

vol_im = CloudVolume(ng_path)
vol_im.chunk_size
vol_im.resolution

Vec(500,500,300, dtype=int64)

In [36]:

ng_path = "precomputed://file:///Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/sample/ng/traces"

info = CloudVolume.create_new_info(
    num_channels=1,
    layer_type="segmentation",
    data_type="uint16",  # Channel images might be 'uint8'
    # raw, png, jpeg, compressed_segmentation, fpzip, kempressed, zfpc, compresso
    encoding="raw",
    resolution=vol_im.resolution,  # Voxel scaling, units are in nanometers
    voxel_offset=[0, 0, 0],  # x,y,z offset in voxels from the origin
    # Pick a convenient size for your underlying chunk representation
    # Powers of two are recommended, doesn't need to cover image exactly
    chunk_size=vol_im.chunk_size,  # units are voxels
    volume_size=vol_im.shape[:3],  # e.g. a cubic millimeter dataset
    skeletons='skeletons'
)
vol = CloudVolume(ng_path, info=info, compress=False)
vol.commit_info()

In [37]:
vol.skeleton.meta.commit_info() #commit skeleton info file

In [63]:
vertices = np.array([[0,0,0], [30,400,400]])
vertices = np.multiply(vertices, vol.resolution)

skel = Skeleton(segid = 1, vertices=vertices, edges=[[0,1]], space='voxel')
print(skel.extra_attributes)
skel.extra_attributes = [ attr for attr in skel.extra_attributes if attr['data_type'] == 'float32' ]
print(skel.extra_attributes)
vol.skeleton.upload(skel)

[{'id': 'radius', 'data_type': 'float32', 'num_components': 1}, {'id': 'vertex_types', 'data_type': 'uint8', 'num_components': 1}]
[{'id': 'radius', 'data_type': 'float32', 'num_components': 1}]


In [61]:
vol.viewer()

Neuroglancer server listening to http://localhost:1337


127.0.0.1 - - [06/Oct/2022 15:17:14] "OPTIONS /info HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2022 15:17:14] "GET /info HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2022 15:17:14] "OPTIONS /skeletons/info HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2022 15:17:14] "GET /skeletons/info HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2022 15:17:14] "OPTIONS /skeletons/1 HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2022 15:17:14] "GET /skeletons/1 HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2022 15:17:14] "OPTIONS /skeletons/4 HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2022 15:17:14] "OPTIONS /500_500_300/0-38_0-500_0-500 HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2022 15:17:14] "GET /skeletons/4 HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2022 15:17:14] code 404, message /500_500_300/0-38_0-500_0-500: Not Found
127.0.0.1 - - [06/Oct/2022 15:17:14] "GET /500_500_300/0-38_0-500_0-500 HTTP/1.1" 404 -
127.0.0.1 - - [06/Oct/2022 15:17:29] "OPTIONS /info HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2022 15:17:29] "GET /info HTTP/1.1" 200 -
127.0.0.1 - - [0


Continuing program execution...
